## Imports

In [1]:
import os
import pandas as pd
import json
import re
from deep_translator import GoogleTranslator
import swifter
import sys
sys.path.append('../helpers_python')
from pre_processing import *

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/robinjaccard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pd.options.display.max_colwidth = 500

## Load the dataset

In [3]:
# import the censored tweets
dfs = []
for r, d, f in os.walk("../"):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("../data/censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [4]:
df_cen.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status', 'quoted_status_permalink', 'is_quote_status',
       'extended_tweet', 'quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level',
       'lang', 'timestamp_ms', 'linked', 'display_text_range',
       'withheld_in_countries', 'extended_entities', 'possibly_sensitive',
       'retweeted_status', 'withheld_copyright'],
      dtype='object')

In [5]:
df_cen.set_index('id', inplace=True)

### Preclean dataset

In [6]:
df_cen.shape

(41727, 38)

In [7]:
df_without_duplicate = df_cen.drop_duplicates("text") # remove duplicates

In [8]:
df_without_duplicate.shape

(23081, 38)

In [9]:
clean_t = clean_tweets(df_without_duplicate["text"]) # apply pre-processing

In [10]:
df_clean = df_without_duplicate.copy()

In [11]:
df_clean['text'] = clean_t

### Remove empty tweets

In [12]:
df_clean[clean_t.apply(lambda x: x.isspace())].shape # number of empty tweets

(725, 38)

In [13]:
df = df_clean.drop(df_clean[df_clean.text.apply(lambda x: x.isspace())].index) # remove empty tweets

### Translation

In [ ]:
translated = df.text.swifter.apply(lambda x: GoogleTranslator(source='auto', target='en').translate(str(x)))

In [ ]:
df['translated'] = translated